# Solutions to PS1

## Problem 1: Tensors and Automatic Differentiation

In [1]:
# Import Packages 
import torch

### 1a)

In [2]:
# Create the first tensor 
a = torch.tensor([1.0, 2.0, 3.0, 4.0, 5.0])
print(a)

#Create the second tensor 
B = torch.ones(3,3)
print(B)

tensor([1., 2., 3., 4., 5.])
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])


In [3]:
# Reshape t1 
a_r = a.reshape(5,1)
print(a_r)

tensor([[1.],
        [2.],
        [3.],
        [4.],
        [5.]])


In [4]:
# Square a 
a_sq = a **2 
print(a_sq)

tensor([ 1.,  4.,  9., 16., 25.])


In [5]:
# Matrix Product of B
B_B = B@B
print(B_B)

tensor([[3., 3., 3.],
        [3., 3., 3.],
        [3., 3., 3.]])


### 1b)

In [6]:
# Computing the differential of f(x) analytically: 
# f'(x) = 2x+3
# f'(2) = 7

In [7]:
# Creating tensor and computing y 
x = torch.tensor([2.0], requires_grad=True)
y = x ** 2 + 3*x + 1

In [8]:
# Call y backwards and compute the gradient 
y.backward()

In [9]:
# Print the Computed Gradient
print(x.grad.item())

7.0


### 1c)

In [ ]:
# Computing the partial differentials of g(x,y) analytically: 
# dg/dx = 2xy, at (1,2) is 4
# dy/dy = x**2 + 3y**2, at (1,2) is 13

In [10]:
# Compute both partial derivatives at (1,2)
x = torch.tensor([1.0], requires_grad=True)
y = torch.tensor([2.0], requires_grad=True)
g = x**2 * y +y ** 3
g.backward()

print(x.grad.item())
print(y.grad.item())

4.0
13.0
